# Abundance Profiles by Region

In [1]:
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

In [2]:
from multiprocessing import Pool
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import binned_statistic
import yaml

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/setuptools/_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [3]:
from auriga.parser import parse
from auriga.images import figure_setup
from auriga.snapshot import Snapshot
from auriga.settings import Settings
from auriga.physics import Physics

In [4]:
RXY_RANGE: tuple = (0, 30)
N_BINS: int = 30
CONFIG_FILE: str = "02"

In [5]:
figure_setup()
settings = Settings()
config = yaml.safe_load(open(f"../configs/{CONFIG_FILE}.yml"))
sample = [f"au{i}_or_l4_s127" for i in settings.groups["Included"]]

In [10]:
def read_data(simulation: str, abundances: list, config: dict) -> tuple:
    """
    This method returns data of interest for this analysis.

    Parameters
    ----------
    simulation : str
        The simulation to consider.
    abundances : tuple
        A list of tuples with the abundances to calculate.
    config : dict
        The configurations dictionary.

    Returns
    -------
    pd.DataFrame
        A data frame with the properties.
    """

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=config["DISC_STD_CIRC"],
        disc_min_circ=config["DISC_MIN_CIRC"],
        cold_disc_delta_circ=config["COLD_DISC_DELTA_CIRC"],
        bulge_max_specific_energy=config["BULGE_MAX_SPECIFIC_ENERGY"])
    s.calculate_gas_temperature()
    s.calculate_hydrogen_number_density()

    for of, to in abundances:
        s.add_metal_abundance(of, to)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_gas = (s.type == 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)
    return_mask = (is_real_star | is_gas) & is_main_obj

    data = {
        "CylindricalRadius_ckpc": s.rho[return_mask],
        "SphericalRadius_ckpc": s.r[return_mask],
        "ComponentTag": s.region_tag[return_mask],
        "HydrogenNumberDensity": s.h_number_density[return_mask],
        "Temperature_K": s.temperature[return_mask],
        "ParticleType": s.type[return_mask],
    }

    for of, to in abundances:
        data[f"[{of}/{to}]"] = s.metal_abundance[f"{of}/{to}"][return_mask]
        data[f"[{of}/{to}]"][~np.isfinite(data[f"[{of}/{to}]"])] = np.nan

    data = pd.DataFrame(data)

    for of, to in abundances:
        data = data[data[f"[{of}/{to}]"].notna()]

    return data

In [64]:
def create_data_dictionary(simulations: list, abundances: list, config: dict):
    physics = Physics()
    settings = Settings()

    dfs = {}
    for simulation in simulations:
        df = read_data(simulation, abundances, config)

        # Calculate binned stats for this dataframe
        stats = {}
        for of, to in abundances:
            stat, bin_edges, _ = binned_statistic(
                x=df["CylindricalRadius_ckpc"][df["ParticleType"] == 4],
                values=df[f"[{of}/{to}]"][df["ParticleType"] == 4],
                statistic=np.nanmean, bins=N_BINS, range=RXY_RANGE)
            stats[f"[{of}/{to}]_Stars"] = stat
            stats[f"BinCenters_ckpc"] = bin_edges[1:] - np.diff(bin_edges)[0] / 2

            stats[f"[{of}/{to}]_Gas"] = binned_statistic(
                x=df["CylindricalRadius_ckpc"][df["ParticleType"] == 0],
                values=df[f"[{of}/{to}]"][df["ParticleType"] == 0],
                statistic=np.nanmean, bins=N_BINS, range=RXY_RANGE)[0]
            stats[f"[{of}/{to}]_ColdGas"] = binned_statistic(
                x=df["CylindricalRadius_ckpc"][(df["ParticleType"] == 0) \
                    & (df["Temperature_K"] <= physics.critical_temperature)],
                values=df[f"[{of}/{to}]"][(df["ParticleType"] == 0) \
                    & (df["Temperature_K"] <= physics.critical_temperature)],
                statistic=np.nanmean, bins=N_BINS, range=RXY_RANGE)[0]
            stats[f"[{of}/{to}]_StarFormingGas"] = binned_statistic(
                x=df["CylindricalRadius_ckpc"][(df["ParticleType"] == 0) \
                    & (df["HydrogenNumberDensity"] \
                        >= physics.star_forming_density)],
                values=df[f"[{of}/{to}]"][(df["ParticleType"] == 0) \
                    & (df["HydrogenNumberDensity"] \
                        >= physics.star_forming_density)],
                statistic=np.nanmean, bins=N_BINS, range=RXY_RANGE)[0]
            
            for i, component in enumerate(settings.components):
                is_component = df["ComponentTag"] == i
                is_star = df["ParticleType"] == 4
                stats[f"[{of}/{to}]_Stars_{component}"] = binned_statistic(
                        x=df["CylindricalRadius_ckpc"][is_star & is_component],
                        values=df[f"[{of}/{to}]"][is_star & is_component],
                        statistic=np.nanmean, bins=N_BINS, range=RXY_RANGE)[0]

        dfs[simulation] = pd.DataFrame(stats)

    return dfs

## Calculate & Save Data

In [65]:
# Calculate data
dfs = create_data_dictionary(
    simulations=sample,
    abundances=[("Fe", 'H'), ('O', 'H')],
    config=config)

In [67]:
# Save data to disk
for key in dfs:
    df = dfs[key]
    df.to_csv(f"../results/{'_'.join(key.split('_')[:-1])}/"
              f"abundance_metallicity{config['FILE_SUFFIX']}.csv")

## Figures

### Stars: [Fe/H] Profiles

In [68]:
fig = plt.figure(figsize=(7, 8))
gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flatten():
    ax.tick_params(which='both', direction="in")
    if ax == axs[-1, -1]: ax.axis("off")
    ax.set_xlim(RXY_RANGE)
    ax.set_ylim(-1.5, 0.5),
    ax.set_xticks([5, 10, 15, 20, 25])
    ax.set_yticks([-1, -0.5, 0])
    ax.grid(True, ls='-', lw=0.25, c="gainsboro")
    if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
        ax.set_xlabel(r"$r_{xy}$ [ckpc]")
        ax.tick_params(labelbottom=True)
    if ax.get_subplotspec().is_first_col():
        ax.set_ylabel("[Fe/H]")


for i, simulation in enumerate(sample):
    galaxy = parse(simulation)[0]
    ax = axs.flatten()[i]
    df = dfs[simulation]
    ax.plot(
        df["BinCenters_ckpc"][np.isfinite(df["[Fe/H]_Stars"])],
        df["[Fe/H]_Stars"][np.isfinite(df["[Fe/H]_Stars"])],
        lw=1.0, zorder=10, c="black", label="All")
    for j, component in enumerate(settings.components):
        color = settings.component_colors[component]
        label = settings.component_labels[component]
        ax.plot(
            df["BinCenters_ckpc"][
                np.isfinite(df[f"[Fe/H]_Stars_{component}"])],
            df[f"[Fe/H]_Stars_{component}"][
                np.isfinite(df[f"[Fe/H]_Stars_{component}"])],
            lw=1.0, zorder=10, c=color, label=label)
    ax.text(x=0.95, y=0.95, size=6.0,
            s=r"$\texttt{" + f"Au{galaxy}" + r"}$",
            ha="right", va="top", transform=ax.transAxes)

axs[1, 0].legend(loc="lower left", framealpha=0.0, fontsize=5.0, ncol=2)

fig.savefig(
    f"../images/FeH_metallicity_profiles_by_region/"
    f"included{config['FILE_SUFFIX']}.pdf")
plt.close(fig)

### Gas: [O/H] Profiles

In [69]:
fig = plt.figure(figsize=(7, 8))
gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flatten():
    ax.tick_params(which='both', direction="in")
    if ax == axs[-1, -1]: ax.axis("off")
    ax.set_xlim(RXY_RANGE)
    ax.set_ylim(-0.5, 1.5),
    ax.set_xticks([5, 10, 15, 20, 25])
    ax.set_yticks([0, 0.5, 1])
    ax.grid(True, ls='-', lw=0.25, c="gainsboro")
    if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
        ax.set_xlabel(r"$r_{xy}$ [ckpc]")
        ax.tick_params(labelbottom=True)
    if ax.get_subplotspec().is_first_col():
        ax.set_ylabel("[O/H]")

for i, simulation in enumerate(sample):
    galaxy = parse(simulation)[0]
    ax = axs.flatten()[i]
    df = dfs[simulation]
    ax.plot(
        df["BinCenters_ckpc"][np.isfinite(df["[O/H]_Gas"])],
        df["[O/H]_Gas"][np.isfinite(df["[O/H]_Gas"])],
        lw=1.0, zorder=10, c="black", label="All")
    ax.plot(
        df["BinCenters_ckpc"][np.isfinite(df["[O/H]_StarFormingGas"])],
        df["[O/H]_StarFormingGas"][np.isfinite(df["[O/H]_StarFormingGas"])],
        lw=1.0, zorder=10, c="red", label="Star-Forming Gas")
    ax.plot(
        df["BinCenters_ckpc"][np.isfinite(df["[O/H]_ColdGas"])],
        df["[O/H]_ColdGas"][np.isfinite(df["[O/H]_ColdGas"])],
        lw=1.0, zorder=10, c="blue", label="Cold Gas")

    ax.text(x=0.95, y=0.95, size=6.0,
            s=r"$\texttt{" + f"Au{galaxy}" + r"}$",
            ha="right", va="top", transform=ax.transAxes)

axs[0, 0].legend(loc="lower left", framealpha=0.0, fontsize=5.0)

fig.savefig(
    f"../images/OH_metallicity_profiles/"
    f"gas_included{config['FILE_SUFFIX']}.pdf")
plt.close(fig)

### Stars: Average [Fe/H] Profiles

In [70]:
# Calculate averages for [Fe/H] stellar abundance
averages = {"BinCenters_ckpc": dfs["au6_or_l4_s127"]["BinCenters_ckpc"].to_numpy()}

for c in settings.components:
    data = np.zeros((len(dfs["au6_or_l4_s127"]), len(dfs)))
    for i, key in enumerate(dfs.keys()):
        data[:, i] = dfs[key][f"[Fe/H]_Stars_{c}"].to_numpy()
        averages[f"[Fe/H]_Stars_{c}"] = np.nanmean(data, axis=1)

averages = pd.DataFrame(averages)

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  


In [79]:
fig = plt.figure(figsize=(7.4, 2.0))
gs = fig.add_gridspec(nrows=1, ncols=4, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flat:
    ax.tick_params(which='both', direction="in")
    ax.set_xlim(RXY_RANGE)
    ax.set_ylim(-1.5, 0.5)
    ax.set_xticks([0, 5, 10, 15, 20, 25])
    ax.set_ylabel("[Fe/H]")
    ax.set_xlabel(r"$r_{xy}$ [ckpc]")
    ax.grid(True, ls='-', lw=0.25, c="gainsboro")
    ax.label_outer()

for idx, component in enumerate(settings.components):
    ax = axs[idx]
    for simulation in sample:
        df = dfs[simulation]
        ax.plot(
            df["BinCenters_ckpc"][
                np.isfinite(df[f"[Fe/H]_Stars_{component}"])],
            df[f"[Fe/H]_Stars_{component}"][
                np.isfinite(df[f"[Fe/H]_Stars_{component}"])],
            lw=1.0, color="silver", zorder=10)
    ax.plot(
        averages["BinCenters_ckpc"],
        averages[f"[Fe/H]_Stars_{component}"],
        lw=1.5, color=settings.component_colors[component], zorder=11)
    axs[idx].text(
        x=0.05, y=0.05, size=8.0, ha='left', va='bottom',
        s=r"$\textbf{" + settings.component_labels[component] + "}$",
        c=settings.component_colors[component],
        transform=axs[idx].transAxes)

fig.savefig(
    f"../images/FeH_metallicity_profiles_by_region/"
    f"included_average{config['FILE_SUFFIX']}.pdf")
plt.close(fig)